In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [75]:
def browser():
    #Initializing the webdriver
    options = webdriver.ChromeOptions()

    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')

    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Users/joe/Sprint2_MMA460/scraping-glassdoor-selenium-master/chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    print('Initialized browser')

In [76]:
def sign_in():

    url = 'https://www.glassdoor.com/profile/login_input.htm'
    browser.get(url)
    
    time.sleep(2)

    # import pdb;pdb.set_trace()

    email_field = browser.find_element_by_name('username')
    password_field = browser.find_element_by_name('password')
    submit_btn = browser.find_element_by_xpath('//button[@type="submit"]')

    email_field.send_keys('pejok98777@hxqmail.com')
    password_field.send_keys('gD1234567Gd')
    submit_btn.click()

    time.sleep(1)
    
    print('Signed In')

In [87]:
def get_jobs_GD(keyword, num_jobs, verbose):
    # BROWSER
    #Initializing the webdriver
    options = webdriver.ChromeOptions()

    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')

    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Users/joe/Sprint2_MMA460/scraping-glassdoor-selenium-master/chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    print('Initialized browser')
    
    # -=-=-=-=-=-=-
    
    #SIGNING IN 
    url = 'https://www.glassdoor.com/profile/login_input.htm'
    driver.get(url)
    
    time.sleep(2)

    # import pdb;pdb.set_trace()

    email_field = driver.find_element_by_name('username')
    password_field = driver.find_element_by_name('password')
    submit_btn = driver.find_element_by_xpath('//button[@type="submit"]')

    email_field.send_keys('pejok98777@hxqmail.com')
    password_field.send_keys('gD1234567Gd')
    submit_btn.click()

    time.sleep(1)
    
    print('Signed In')
    
    # -=-=-=-=-=-=-
    
    # DATA MINING SECTION
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=S&locId=1282&locKeyword=North%20Carolina&jobType=all&fromAge=30&minSalary=0&includeNoSalaryJobs=true&radius=25&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    
    driver.get(url)
    jobs = []
    
    print('Opened Page')
        
        
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click() 
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_class_name("SVGInline modal_closeIcon").click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [91]:
#This line will open a new chrome window and start the scraping.
df = get_jobs_GD("analyst", 1200, False)
df

Initialized browser
Signed In
Opened Page
Progress: 0/1200
Progress: 1/1200
Progress: 2/1200
Progress: 3/1200
Progress: 4/1200
Progress: 5/1200
Progress: 6/1200
Progress: 7/1200
Progress: 8/1200
Progress: 9/1200
Progress: 10/1200
Progress: 11/1200
Progress: 12/1200
Progress: 13/1200
Progress: 14/1200
Progress: 15/1200
Progress: 16/1200
Progress: 17/1200
Progress: 18/1200
Progress: 19/1200
Progress: 20/1200
Progress: 21/1200
Progress: 22/1200
Progress: 23/1200
Progress: 24/1200
Progress: 25/1200
Progress: 26/1200
Progress: 27/1200
Progress: 28/1200
Progress: 29/1200
Progress: 30/1200
Progress: 31/1200
Progress: 32/1200
Progress: 33/1200
Progress: 34/1200
Progress: 35/1200
Progress: 36/1200
Progress: 37/1200
Progress: 38/1200
Progress: 39/1200
Progress: 40/1200
Progress: 41/1200
Progress: 42/1200
Progress: 43/1200
Progress: 44/1200
Progress: 45/1200
Progress: 46/1200
Progress: 47/1200
Progress: 48/1200
Progress: 49/1200
Progress: 50/1200
Progress: 51/1200
Progress: 52/1200
Progress: 53/1

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".selected"}
  (Session info: chrome=80.0.3987.132)


In [1]:
get_jobs('analyst', 10, True)

NameError: name 'get_jobs' is not defined

In [93]:
df.head(158)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,-1,About Agari\n\n\nFounded by thought leaders of...,4.2,Agari\n4.2,"Raleigh, NC","Foster City, CA",51 to 200 employees,2009,Company - Private,Enterprise Software & Network Solutions,Information Technology,$25 to $50 million (USD),-1
1,Senior Data Scientist - Threat Intelligence Re...,-1,What you’ll be doing...\n\nThis role requires ...,3.7,Verizon\n3.7,"Cary, NC","New York, NY",10000+ employees,2000,Company - Public,Telecommunications Services,Telecommunications,$10+ billion (USD),"Amazon, Microsoft, Google"
2,Entry Level Data Scientist,-1,The Center for\nData Science at RTI Internatio...,4.3,RTI International\n4.3,"Durham, NC","Research Triangle Park, NC",1001 to 5000 employees,1958,Nonprofit Organization,Research & Development,Business Services,$500 million to $1 billion (USD),"Westat, Abt Associates, Chemonics International"
3,Data Scientist I,-1,LexisNexis is looking for a Data Scientist I.\...,3.5,Elsevier\n3.5,"Raleigh, NC","Amsterdam, Netherlands",5001 to 10000 employees,1880,Subsidiary or Business Segment,Publishing,Media,$2 to $5 billion (USD),"Clarivate Analytics, Springer Nature, Thomson ..."
4,Data Scientist Intern,-1,Infinia ML is looking for interns excited to d...,4.4,Infinia ML\n4.4,"Durham, NC","Durham, NC",1 to 50 employees,2017,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
5,Data Scientist,-1,"As a Data Scientist, you will evaluate and imp...",3.5,Quadient\n3.5,"Raleigh, NC","Bagneux, France",5001 to 10000 employees,1924,Company - Public,Enterprise Software & Network Solutions,Information Technology,$1 to $2 billion (USD),"Thunderhead, OpenText, Acquia"
6,Applied Data Scientist,-1,THE POSITION\n\nLendingTree's Data Science tea...,3.8,LendingTree\n3.8,"Charlotte, NC","Charlotte, NC",1001 to 5000 employees,1996,Company - Public,Lending,Finance,$500 million to $1 billion (USD),"Zillow, Trulia"
7,Business Analytics Data Scientist,-1,"Thursday, April 25, 2019\n\nPosition: Business...",4.6,Carolina Panthers\n4.6,"Charlotte, NC","Charlotte, NC",51 to 200 employees,1989,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$10 to $25 million (USD),-1
8,Data Scientist,-1,growing rapidly and looking for exceptional ne...,4.8,Auth0\n4.8,"Charlotte, NC","Bellevue, WA",501 to 1000 employees,2014,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1
9,Data Scientist (All Levels),-1,Help build technology that saves lives!\nLeanT...,4.5,LeanTaaS\n4.5,"Charlotte, NC","Santa Clara, CA",51 to 200 employees,2010,Company - Private,Health Care Services & Hospitals,Health Care,$25 to $50 million (USD),-1


In [101]:
#analyst_150 = get_jobs_GD("analyst", 150, False)
analyst_150.to_csv ('analyst_150_3-8.csv',float_format="%1.2f")
analyst_150

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Test Analyst - US,-1,Test Analyst--SOA/API\n\n03/06/2020\n\nJob Des...,3.1,Infosys\n3.1,"Raleigh, NC","Bengaluru, India",10000+ employees,1981,Company - Public,IT Services,Information Technology,$10+ billion (USD),"Tata Consultancy Services, Accenture, Cognizan..."
1,Jr. Analyst,-1,At Dominion Energy we love our jobs. That’s ri...,3.8,Dominion Energy\n3.8,"Gastonia, NC","Richmond, VA",10000+ employees,1909,Company - Public,Energy,"Oil, Gas, Energy & Utilities",$10+ billion (USD),-1
2,Network Analyst,-1,Sampson Regional Medical Center is hiring a Ne...,2.2,Sampson Regional Medical Center\n2.2,"Clinton, NC","Clinton, NC",501 to 1000 employees,1949,Hospital,Health Care Services & Hospitals,Health Care,$50 to $100 million (USD),-1
3,Executive Analyst,-1,"Under the direction of the CFO, this position ...",3.8,Bob Barker Co.\n3.8,"Fuquay-Varina, NC","Fuquay Varina, NC",201 to 500 employees,1972,Company - Private,Wholesale,Business Services,$100 to $500 million (USD),-1
4,BUSINESS ANALYST,-1,SUMMARY: Business Analyst will leverage their ...,2.4,KGPCo\n2.4,"Raleigh, NC","Faribault, MN",1001 to 5000 employees,1973,Company - Private,Telecommunications Services,Telecommunications,Unknown / Non-Applicable,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Data Analyst,-1,Job Description\nThis position will focus on s...,-1,TRC Staffing - Kennesaw,"Greensboro, NC",-1,-1,-1,-1,-1,-1,-1,-1
146,Business Analyst,-1,Business Analyst\n\nAbout Us:\n\n\nTransportat...,5.0,Transportation Impact\n5.0,"Emerald Isle, NC","Emerald Isle, NC",1 to 50 employees,-1,Company - Private,-1,-1,$1 to $5 million (USD),-1
147,Business Analyst,-1,Job Description\n\nJob #: 1053562\nJob Title: ...,3.9,Apex Systems\n3.9,"Charlotte, NC","Glen Allen, VA",1001 to 5000 employees,1995,Subsidiary or Business Segment,Staffing & Outsourcing,Business Services,$2 to $5 billion (USD),"TEKsystems, Insight Global, Accenture"
148,QA Analyst,-1,"At TrialCard, our passion is success - success...",3.4,TrialCard\n3.4,"Morrisville, NC","Morrisville, NC",1001 to 5000 employees,2001,Company - Private,Advertising & Marketing,Business Services,$50 to $100 million (USD),-1


In [104]:
analyst_500 = get_jobs_GD("analyst", 500, False)
analyst_500.to_csv ('analyst_500_3-8.csv',float_format="%1.2f")
analyst_500

Initialized browser
Signed In
Opened Page
Progress: 0/500
Progress: 1/500
Progress: 2/500
Progress: 3/500
Progress: 4/500
Progress: 5/500
Progress: 6/500
Progress: 7/500
Progress: 8/500
Progress: 9/500
Progress: 10/500
Progress: 11/500
Progress: 12/500
Progress: 13/500
Progress: 14/500
Progress: 15/500
Progress: 16/500
Progress: 17/500
Progress: 18/500
Progress: 19/500
Progress: 20/500
Progress: 21/500
Progress: 22/500
Progress: 23/500
Progress: 24/500
Progress: 25/500
Progress: 26/500
Progress: 27/500
Progress: 28/500
Progress: 29/500
Progress: 30/500
Progress: 31/500
Progress: 32/500
Progress: 33/500
Progress: 34/500
Progress: 35/500
Progress: 36/500
Progress: 37/500
Progress: 38/500
Progress: 39/500
Progress: 40/500
Progress: 41/500
Progress: 42/500
Progress: 43/500
Progress: 44/500
Progress: 45/500
Progress: 46/500
Progress: 47/500
Progress: 48/500
Progress: 49/500
Progress: 50/500
Progress: 51/500
Progress: 52/500
Progress: 53/500
Progress: 54/500
Progress: 55/500
Progress: 56/500

Progress: 459/500
Progress: 460/500
Progress: 461/500
Progress: 462/500
Progress: 463/500
Progress: 464/500
Progress: 465/500
Progress: 466/500
Progress: 467/500
Progress: 468/500
Progress: 469/500
Progress: 470/500
Progress: 471/500
Progress: 472/500
Progress: 473/500
Progress: 474/500
Progress: 475/500
Progress: 476/500
Progress: 477/500
Progress: 478/500
Progress: 479/500
Progress: 480/500
Progress: 481/500
Progress: 482/500
Progress: 483/500
Progress: 484/500
Progress: 485/500
Progress: 486/500
Progress: 487/500
Progress: 488/500
Progress: 489/500
Progress: 490/500
Progress: 491/500
Progress: 492/500
Progress: 493/500
Progress: 494/500
Progress: 495/500
Progress: 496/500
Progress: 497/500
Progress: 498/500
Progress: 499/500
Progress: 500/500


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Test Analyst - US,-1,Test Analyst--SOA/API\n\n03/06/2020\n\nJob Des...,3.1,Infosys\n3.1,"Raleigh, NC","Bengaluru, India",10000+ employees,1981,Company - Public,IT Services,Information Technology,$10+ billion (USD),"Tata Consultancy Services, Accenture, Cognizan..."
1,Executive Analyst,-1,"Under the direction of the CFO, this position ...",3.8,Bob Barker Co.\n3.8,"Fuquay-Varina, NC","Fuquay Varina, NC",201 to 500 employees,1972,Company - Private,Wholesale,Business Services,$100 to $500 million (USD),-1
2,Jr. Analyst,-1,At Dominion Energy we love our jobs. That’s ri...,3.8,Dominion Energy\n3.8,"Gastonia, NC","Richmond, VA",10000+ employees,1909,Company - Public,Energy,"Oil, Gas, Energy & Utilities",$10+ billion (USD),-1
3,Support Analyst - Enterprise,-1,Overview\n\n\nGreat software requires hard wor...,3.6,Esri\n3.6,"Charlotte, NC","Redlands, CA",1001 to 5000 employees,1969,Company - Private,Computer Hardware & Software,Information Technology,$1 to $2 billion (USD),Pitney Bowes
4,BUSINESS ANALYST,-1,SUMMARY: Business Analyst will leverage their ...,2.4,KGPCo\n2.4,"Raleigh, NC","Faribault, MN",1001 to 5000 employees,1973,Company - Private,Telecommunications Services,Telecommunications,Unknown / Non-Applicable,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Infrastructure/Application Business Analyst,-1,Job Description: The business analyst role ana...,4.0,Peterson Technology Partners\n4.0,"Charlotte, NC","Park Ridge, IL",201 to 500 employees,1997,Contract,Computer Hardware & Software,Information Technology,$25 to $50 million (USD),-1
496,Principal Programmer Analyst/ Principal Progra...,-1,LI-KF1\n#GD\n\nPPD’s mission is to improve hea...,3.6,Pharmaceutical Product Development\n3.6,"Wilmington, NC","Wilmington, NC",10000+ employees,1985,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),-1
497,Procurement Analyst,-1,Company:\n\nElior North America\n\nPosition:\n...,3.2,Elior North America\n3.2,"Charlotte, NC","Courbevoie, France",10000+ employees,1991,Subsidiary or Business Segment,Catering & Food Service Contractors,"Restaurants, Bars & Food Services",Unknown / Non-Applicable,-1
498,"IT Business Analyst, Manufacturing",-1,"IT Business Analyst, Manufacturing\nLocation\n...",3.5,Butterball\n3.5,"Garner, NC","Garner, NC",5001 to 10000 employees,2006,Company - Private,Food & Beverage Manufacturing,Manufacturing,$2 to $5 billion (USD),-1


In [106]:
analyst_all = get_jobs_GD("analyst", 3790, False)
analyst_all.to_csv ('analyst_all_3-8.csv',float_format="%1.2f")
analyst_all

Initialized browser
Signed In
Opened Page
Progress: 0/3790
Progress: 1/3790
Progress: 2/3790
Progress: 3/3790
Progress: 4/3790
Progress: 5/3790
Progress: 6/3790
Progress: 7/3790
Progress: 8/3790
Progress: 9/3790
Progress: 10/3790
Progress: 11/3790
Progress: 12/3790
Progress: 13/3790
Progress: 14/3790
Progress: 15/3790
Progress: 16/3790
Progress: 17/3790
Progress: 18/3790
Progress: 19/3790
Progress: 20/3790
Progress: 21/3790
Progress: 22/3790
Progress: 23/3790
Progress: 24/3790
Progress: 25/3790
Progress: 26/3790
Progress: 27/3790
Progress: 28/3790
Progress: 29/3790
Progress: 30/3790
Progress: 31/3790
Progress: 32/3790
Progress: 33/3790
Progress: 34/3790
Progress: 35/3790
Progress: 36/3790
Progress: 37/3790
Progress: 38/3790
Progress: 39/3790
Progress: 40/3790
Progress: 41/3790
Progress: 42/3790
Progress: 43/3790
Progress: 44/3790
Progress: 45/3790
Progress: 46/3790
Progress: 47/3790
Progress: 48/3790
Progress: 49/3790
Progress: 50/3790
Progress: 51/3790
Progress: 52/3790
Progress: 53/3

Progress: 435/3790
Progress: 436/3790
Progress: 437/3790
Progress: 438/3790
Progress: 439/3790
Progress: 440/3790
Progress: 441/3790
Progress: 442/3790
Progress: 443/3790
Progress: 444/3790
Progress: 445/3790
Progress: 446/3790
Progress: 447/3790
Progress: 448/3790
Progress: 449/3790
Progress: 450/3790
Progress: 451/3790
Progress: 452/3790
Progress: 453/3790
Progress: 454/3790
Progress: 455/3790
Progress: 456/3790
Progress: 457/3790
Progress: 458/3790
Progress: 459/3790
Progress: 460/3790
Progress: 461/3790
Progress: 462/3790
Progress: 463/3790
Progress: 464/3790
Progress: 465/3790
Progress: 466/3790
Progress: 467/3790
Progress: 468/3790
Progress: 469/3790
Progress: 470/3790
Progress: 471/3790
Progress: 472/3790
Progress: 473/3790
Progress: 474/3790
Progress: 475/3790
Progress: 476/3790
Progress: 477/3790
Progress: 478/3790
Progress: 479/3790
Progress: 480/3790
Progress: 481/3790
Progress: 482/3790
Progress: 483/3790
Progress: 484/3790
Progress: 485/3790
Progress: 486/3790
Progress: 48

Progress: 867/3790
Progress: 868/3790
Progress: 869/3790
Progress: 870/3790
Progress: 871/3790
Progress: 872/3790
Progress: 873/3790
Progress: 874/3790
Progress: 875/3790
Progress: 876/3790
Progress: 877/3790
Progress: 878/3790
Progress: 879/3790
Progress: 880/3790
Progress: 881/3790
Progress: 882/3790
Progress: 883/3790
Progress: 884/3790
Progress: 885/3790
Progress: 886/3790
Progress: 887/3790
Progress: 888/3790
Progress: 889/3790
Progress: 890/3790
Progress: 891/3790
Progress: 892/3790
Progress: 893/3790
Progress: 894/3790
Progress: 895/3790
Progress: 896/3790
Progress: 897/3790
Progress: 898/3790
Progress: 899/3790
Progress: 900/3790
Progress: 901/3790
Progress: 902/3790
Progress: 903/3790
Progress: 904/3790
Progress: 905/3790
Progress: 906/3790
Progress: 907/3790
Progress: 908/3790
Progress: 909/3790
Progress: 910/3790
Progress: 911/3790
Progress: 912/3790
Progress: 913/3790
Progress: 914/3790
Progress: 915/3790
Progress: 916/3790
Progress: 917/3790
Progress: 918/3790
Progress: 91

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Executive Analyst,-1,"Under the direction of the CFO, this position ...",3.8,Bob Barker Co.\n3.8,"Fuquay-Varina, NC","Fuquay Varina, NC",201 to 500 employees,1972,Company - Private,Wholesale,Business Services,$100 to $500 million (USD),-1
1,Test Analyst - US,-1,Test Analyst--SOA/API\n\n03/06/2020\n\nJob Des...,3.1,Infosys\n3.1,"Raleigh, NC","Bengaluru, India",10000+ employees,1981,Company - Public,IT Services,Information Technology,$10+ billion (USD),"Tata Consultancy Services, Accenture, Cognizan..."
2,Jr. Analyst,-1,At Dominion Energy we love our jobs. That’s ri...,3.8,Dominion Energy\n3.8,"Gastonia, NC","Richmond, VA",10000+ employees,1909,Company - Public,Energy,"Oil, Gas, Energy & Utilities",$10+ billion (USD),-1
3,Support Analyst - Enterprise,-1,Overview\n\n\nGreat software requires hard wor...,3.6,Esri\n3.6,"Charlotte, NC","Redlands, CA",1001 to 5000 employees,1969,Company - Private,Computer Hardware & Software,Information Technology,$1 to $2 billion (USD),Pitney Bowes
4,BUSINESS ANALYST,-1,SUMMARY: Business Analyst will leverage their ...,2.4,KGPCo\n2.4,"Raleigh, NC","Faribault, MN",1001 to 5000 employees,1973,Company - Private,Telecommunications Services,Telecommunications,Unknown / Non-Applicable,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,Data Quality Analyst,-1,Role/Skills: Data Quality Analyst\nWork Locati...,3.4,Net2Source\n3.4,"Cary, NC","New Brunswick, NJ",1001 to 5000 employees,2007,Company - Private,Staffing & Outsourcing,Business Services,$100 to $500 million (USD),-1
954,Global Payments Business Analyst,-1,RESPONSIBILITIES:\n\nKforce is seeking a Busin...,4.1,Kforce\n4.1,"Winston-Salem, NC","Tampa, FL",10000+ employees,1966,Company - Public,Staffing & Outsourcing,Business Services,$1 to $2 billion (USD),-1
955,System Analyst,-1,Job Title: Technical System Analyst\nLocation:...,3.6,LanceSoft Inc\n3.6,"Charlotte, NC","Herndon, VA",1001 to 5000 employees,2000,Company - Private,Staffing & Outsourcing,Business Services,$50 to $100 million (USD),"Mindlance, Artech Information Systems, Pyramid..."
956,Financial Analyst,-1,Job Description\nWe are currently seeking a Fi...,-1,Anchor Staffing,"Fletcher, NC","East Brunswick, NJ",51 to 200 employees,-1,Company - Public,-1,-1,Unknown / Non-Applicable,-1
